# Ungraded Lab: Building a Custom Dense Layer

In this lab, we'll walk through how to create a custom layer that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) class. Unlike simple Lambda layers you did previously, the custom layer here will contain weights that can be updated during training.

## Imports

In [21]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


## Custom Layer with weights

To make custom layer that is trainable, we need to define a class that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) base class from Keras. The Python syntax is shown below in the class declaration. This class requires three functions: `__init__()`, `build()` and `call()`. These ensure that our custom layer has a *state* and *computation* that can be accessed during training or inference.

In [22]:
a = np.array([[1, 2], [3, 4]])
a = tf.convert_to_tensor(a, dtype=tf.float32)
b = tf.ones((2, 1), dtype=tf.float32)
b.numpy()
tf.matmul(a, b)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[3.],
       [7.]], dtype=float32)>

In [28]:
# inherit from this base class
from tensorflow.keras.layers import Layer

class SimpleDense(Layer):

    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # initialize the weights
        w_init = tf.random_normal_initializer()
        print(f"input_shape = {input_shape}")
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return tf.matmul(inputs, self.w) + self.b

Now we can use our custom layer like below:

In [29]:
# declare an instance of the class
my_dense = SimpleDense(units=1)

# define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense(x)

# parameters of the base Layer class like `variables` can be used
print(my_dense.variables)
print(f"x = {x}")
print(f"y = {y}")
print(f"tf.is_tensor(x) = {tf.is_tensor(x)}")
print(f"tf.is_tensor(y) = {tf.is_tensor(y)}")

input_shape = (1, 1)
[<tf.Variable 'simple_dense_10/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.01261667]], dtype=float32)>, <tf.Variable 'simple_dense_10/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]
x = [[1.]]
y = [[0.01261667]]
tf.is_tensor(x) = True
tf.is_tensor(y) = True


Let's then try using it in simple network:

In [51]:
# use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])

input_shape = (1, 1)
model.build(input_shape)
model.summary()

input_shape = (1, 1)
Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_dense_30 (SimpleDens  (1, 1)                   2         
 e)                                                              
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [53]:
# define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

xs = np.reshape(xs, (xs.size, 1))
ys = np.reshape(ys, (ys.size, 1))

print(f"xs = {xs}")
print(f"ys = {ys}")

print(f"xs.shape = {np.shape(xs)}")
print(f"ys.shape = {np.shape(ys)}")
print(f"xs.shape[-1] = {xs.shape[-1]}")

# use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])

# configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500, verbose=0)

# perform inference
print(model.predict([10.0]))

# see the updated state of the variables
print(my_layer.variables)

xs = [[-1.]
 [ 0.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 4.]]
ys = [[-3.]
 [-1.]
 [ 1.]
 [ 3.]
 [ 5.]
 [ 7.]]
xs.shape = (6, 1)
ys.shape = (6, 1)
xs.shape[-1] = 1
input_shape = (None, 1)


1/1 [==============================] - 0s 71ms/step
[[18.981129]]
[<tf.Variable 'simple_dense_32/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9972651]], dtype=float32)>, <tf.Variable 'simple_dense_32/bias:0' shape=(1,) dtype=float32, numpy=array([-0.9915211], dtype=float32)>]
